# Resources
* [AI-Scientist](https://github.com/SakanaAI/AI-Scientist )

# Init + Env

In [ ]:
from os import path, makedirs
from aa_scrape_PDF import copy_pdfs

environ["_GRG_MODEL_NAME"] = "llama3.2-vision"
environ["_GRG_EMBED_NAME"] = "all-minilm"
